In [2]:
import polars as pl
import requests

def fetch_news_data(stock, start_date, end_date,turn):
    """
    Makes an API request and returns the response data.
    
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: Response JSON as a dictionary or None if the request fails.
    """
    if turn:
        if end_date:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=ZRX2N0ETMX8N4GOR"
        else:
            print("65DCZMTJLB2SERT9")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=MLPIDKVUZKO71B19"
    else:
        if end_date:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=LPHO83KMASND8ZJJ"
        else:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=47WWMCYJI8DEB3KR"
    response = requests.get(endpoint)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

def convert_to_dataframe(news_items):
    """
    Converts a list of news items into a Polars DataFrame and extracts the date.
    
    :param news_items: List of news items.
    :return: A Polars DataFrame with an added 'date' column.
    """
    news_items = news_items.get("feed", [])
    if news_items:
        df = pl.DataFrame(news_items)
        df = df.with_columns(pl.col("time_published").str.slice(0, 8).alias("date"))
        return df
    return pl.DataFrame()

def aggregate_news_data(stocks, start_date, end_date, turn):
    """
    Fetches and aggregates news data for a specified number of API calls.
     
    :param num_calls: Number of API calls to make.
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: A Polars DataFrame containing aggregated news data.
    """
    df_list = []
    for stock in stocks:
        # Fetch the news data
        news_data = fetch_news_data(stock, start_date, end_date, turn)
        
        if news_data:
            # Convert to DataFrame and append to the list
            df = convert_to_dataframe(news_data)
            if df.height == 0:
                print(news_data)
                print('yes')
                print(df)
            df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    return pl.concat(df_list, how='vertical') if df_list else pl.DataFrame()

In [3]:
stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'BRK-A', 'LLY', 'AVGO']
news_data = aggregate_news_data(stocks, 20240422, False, False)

In [7]:
news_data = news_data.to_pandas()
news_data

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,date
0,Futures: Fed Meeting Looms; Bitcoin Leader Sli...,https://www.investors.com/market-trend/stock-m...,20240429T210300,"[SCOTT LEHTONEN, Investor's Business Daily]",Futures: Fed Meeting Looms. Bitcoin Leader Sli...,https://www.investors.com/wp-content/uploads/2...,Investors Business Daily,n/a,www.investors.com,"[{'topic': 'Economy - Fiscal', 'relevance_scor...",0.212316,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.2068...",20240429
1,Stock Market Today: Markets Post Broad-Based G...,https://www.kiplinger.com/investing/stocks/sto...,20240429T200758,[Dan Burrows],"Stocks enjoyed broad-based gains Monday, boost...",https://cdn.mos.cms.futurecdn.net/hwgJ7osrMtUW...,Kiplinger,n/a,www.kiplinger.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.188458,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.0570...",20240429
2,Apple Earnings: Time to Take a Bite?,https://www.zacks.com/commentary/2264463/apple...,20240429T192900,[Derek Lewis],Investor favorite Apple is slated to unveil qu...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",-0.016993,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.2738...",20240429
3,Wall Street to Start a Very Crucial Week - App...,https://www.benzinga.com/news/earnings/24/04/3...,20240429T185732,[Zacks],We have a highly consequential week on tap for...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.108956,Neutral,"[{'ticker': 'DMPZF', 'relevance_score': '0.058...",20240429
4,Will Solid Client & Datacenter Aid AMD's Q1 Ea...,https://www.zacks.com/stock/news/2264427/will-...,20240429T171800,[Zacks Equity Research],AMD's first-quarter 2024 earnings are likely t...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.6...",0.276426,Somewhat-Bullish,"[{'ticker': 'AMD', 'relevance_score': '0.72389...",20240429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,Nvidia's High-End Chips Ended Up In Chinese Ha...,https://www.benzinga.com/markets/asia/24/04/38...,20240423T050111,[Benzinga Neuro],Chinese entities have managed to acquire high-...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Mergers & Acquisitions', 'relevanc...",0.095783,Neutral,"[{'ticker': 'NVDA', 'relevance_score': '0.4274...",20240423
3685,Broadcom Inc. ( AVGO ) Gains But Lags Market...,https://www.zacks.com/stock/news/2259620/broad...,20240422T214516,[Zacks Equity Research],"In the most recent trading session, Broadcom I...",https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.149760,Neutral,"[{'ticker': 'AVGO', 'relevance_score': '0.5372...",20240422
3686,Data Center Interconnect Market size worth $ 3...,https://www.benzinga.com/pressreleases/24/04/g...,20240422T140000,[Globe Newswire],"Jersey City, New Jersey, April 22, 2024 ( GLOB...",https://www.benzinga.com/next-assets/images/sc...,Benzinga,General,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.252395,Somewhat-Bullish,"[{'ticker': 'CIEN', 'relevance_score': '0.0282...",20240422
3687,Is Nvidia Getting Ready to Disrupt Another Com...,https://www.fool.com/investing/2024/04/22/is-n...,20240422T123900,[Nicholas Rossolillo],Some investors were second-guessing the streng...,https://g.foolcdn.com/editorial/images/773500/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.200374,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0584...",20240422


In [8]:
news_data.to_csv('news_data.csv', mode = 'a', index = False)